<a href="https://colab.research.google.com/github/BitLorax/depth-prediction/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm -r .git

In [0]:
!git init
!git remote add -f origin https://github.com/BitLorax/depth-prediction.git

!git config core.sparseCheckout true
!echo 'nyuDepth' >> .git/info/sparse-checkout

In [0]:
!git pull origin master

In [0]:
import tensorflow

In [0]:
import numpy as np
import matplotlib.pyplot as plt